### DocChat 案例介绍
得益于大模型优秀的zero/few shot learning能力，当前可通过将特定Documents((PDF, blog, Notion pages等)作为Context传入模型，从而快速适配特定领域知识并应用,这一场景可大致统一为DocChat。

将原始文档等非结构化数据转化并实现DocChat大致可分为以下几步：

- Loading 加载：首先，我们需要加载数据。非结构化数据可以从多个来源加载。使用LangChain集成中心浏览完整的加载器集合。每个加载器将数据返回为LangChain文档。

- Splitting 分割：文本分割器将文档拆分为指定大小的片段。

- Storage 存储：存储（例如矢量存储）将保存并常常嵌入这些片段。

- Retireval 检索：应用程序从存储中检索片段（通常具有与输入问题相似的嵌入特征）。

- Generation 生成：语言模型生成一个答案，使用的提示包括问题和检索到的数据。

- Conbersation：通过在问答链中添加记忆（Memory），进行多轮对话。

我们使用网络上获得的流程作为演示，它很好的解释了上面的过程。

从不同工具和组件的角度来看：
<div style="text-align: center;">
<img src="./img/langchain+chatglm.png" alt="Image" width="600">
</div>

从文档的角度来看：
<div style="text-align: center;">
<img src="./img/langchain+chatglm2.png" alt="Image" width="600">
</div>


接下来我们将基于langchain复现DocChat的整体流程,这里将侧重于对于langchain关键组件的使用和介绍，默认环境已正确配置。

**Step 1. Load**  
langchain 提供了一系列的工具来支持 CSV, FileDirectory, HTML, Json等格式的文本载入，同时提供一系列集成工具以实现AWS S3, Arxiv, Email, Github， Google BigQuery等100+平台数据接入，具体支持清单可参考[官方文档](https://python.langchain.com/docs/integrations/document_loaders/)

这里我们使用特定的`DocumentLoader`(WebBaseLoader 和 xx ) 将非结构化数据载入为`Documents`, 每个`Document`由一系列文本片段(`page_content`字段)和相关元数据组成。

In [1]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.openeuler.org/zh/blog/20230630-newcomer/0630-newcomer.html")
data = loader.load()
print(data)

[Document(page_content='\n\n\n\n\nopenEuler社区参与之旅\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n博客openEuler社区参与之旅openEuler社区参与之旅openEuler2023-06-30openEuler欢迎来到openEuler！签署CLA #您必须首先签署签署CLA协议，然后才能参与社区贡献。根据您的参与身份，选择签署个人 CLA、员工 CLA 或企业 CLA。协议的签署地址是：https://clasign.osinfra.cn/sign/gitee_openeuler-1611298811283968340\xa0安装openEuler #请参考[下载安装openEuler]开始您的贡献 #签署好协议以后，就需要考虑在社区里具体能做点什么了。参与社区有很多种方法和形式，如果总结起来，大体有下面的三类：1.\xa0\xa0\xa0\xa0\xa0\xa0提交一些需求，或者bug，简单来说就是在用openEuler的过程中发现了一些问题，然后需要在社区把这个问题提出来。2.\xa0\xa0\xa0\xa0\xa0\xa0为社区修正bug，这是更高一个层面的参与社区了，在这个层面，参与者实质上是以一个开发者的姿态进入到了社区中。一般我们都提倡，除了提出问题，更期待大家能解决问题。3.\xa0\xa0\xa0\xa0\xa0\xa0贡献软件包，发现openEuler缺失了一个软件包，帮openEuler把这个软件包补上。实际上贡献软件包的过程就是帮助openEuler提供更丰富功能的过程。希望随着大家的参与，openEuler能够成为一个"无所不有"的软件生态系统。我们就来看看这3种参与方式如何进行吧。在具体讨论参与方法之前，大家可以先保存下面三个网址链接。openEuler官网：https://openeuler.org/openEuler代码仓：\xa0https://gitee.com/openeuler/openEuler软件包仓：https://gitee.com/src-openeuler第一个网址是openEuler的官网，是供大家获取一些通用信息的地方。而真正我们所谓的"社区"则是体现在2，3这两个网址上。提出问题或建议（提Issue） #如果您发现并想向社区上报问题或缺陷，问

**Step 2. Split**   
将`Document`分割为特定大小文本块用于embedding 和 vector storage.

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

**Step 3. Store**  
为了能够查找已拆分的文档，我们首先需要将它们存储在方便查询的位置。最常用的方法是将每个文档的内容 embedding ，然后将 embedding 和文档存储在向量存储中，其中使用 embedding 来索引文档。

In [3]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

**Step 4. Retrieve**  
langchain提供了很多类别的 Retrievers, 包含但不限于 Vectorstores，所有的 Retrievers都实现了 `get_relevant_documents()`和异步版本 `aget_relevant_documents()` 实现文档检索。这里我们使用相似度搜索从 Vectorstores 检索与问题相关的分割的文档块。

In [4]:
question = "如何参与社区活动?"
docs = vectorstore.similarity_search(question)
len(docs)

4

基于距离的向量数据库检索将查询嵌入到高维空间中，并根据"距离"找到类似的嵌入式文档。但是，如果查询措辞稍有变化，或者嵌入式未能很好地捕捉数据的语义，检索结果可能会产生不同的结果。`MultiQueryRetriever`通过使用语言模型（LLM）自动生成多个具有不同角度的查询，从而自动化提示调优过程。对于每个查询，它检索一组相关文档，并在所有查询之间取唯一并集，以获得一个更大的潜在相关文档集合。通过对同一问题生成多个视角，`MultiQueryRetriever`可能能够克服基于距离的检索的某些限制，并获得更丰富的结果集。

In [5]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectorstore.as_retriever(),
                                                  llm=ChatOpenAI(temperature=0))
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

6

**Step 5. Generate**  
在使用langchain过程中数据会频繁的在Loaders, Models, Vectorstores, Agent等组件间流动，针对特定场景对工具的调用和数据传输遵守特定流程，因此langchain提供了`chain`组件实现对于工具的固定编排，从而在特定任务上快速开发。 这里我们使用`RetrievalQA` chain 与LLM/Chat模型（例如gpt-3.5-turbo）将检索到的文档提炼成答案。
注意：由于chat模型在输入输出中通常包含固定格式(普遍为 [{"role": "user", "message": "hello."}...] ), 在langchain中 将 chat模型和 LLM (输入输出均为字符串) 作为不同类型。langchain当前提供55 个chat模型和LLM, 详见[清单](https://python.langchain.com/docs/integrations/llms/)

In [6]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())
qa_chain({"query": question})

{'query': '如何参与社区活动?',
 'result': '参与openEuler社区活动有以下几个步骤：\n\n1. 签署CLA协议：首先，您需要签署CLA协议，才能参与社区贡献。根据您的参与身份，选择签署个人CLA、员工CLA或企业CLA。协议的签署地址是：https://clasign.osinfra.cn/sign/gitee_openeuler-1611298811283968340\n\n2. 下载安装openEuler：请参考[下载安装openEuler]开始您的贡献。\n\n3. 选择参与方式：根据您的兴趣和技能，选择以下三种参与方式之一：\n   - 提交需求或bug：在使用openEuler的过程中，如果发现了问题或有改进建议，可以在社区中提交需求或bug报告。\n   - 修正bug：如果您是开发者，可以参与到社区中，修复已知的bug，提高openEuler的质量。\n   - 贡献软件包：如果发现openEuler缺失了某个软件包，您可以帮助openEuler补充这个软件包，丰富openEuler的功能。\n\n4. 参考网址链接：在具体讨论参与方法之前，您可以保存以下三个网址链接，以便获取更多信息：\n   - openEuler官网：https://openeuler.org/openEuler\n   - openEuler代码仓：https://gitee.com/openeuler/openEuler\n   - 软件包仓：https://gitee.com/src-op\n\n5. 社区内沟通方式：openEuler社区的交流方式包括邮件列表和视频会议等。您可以访问https://www.openeuler.org/zh/community/mailing-list/，找到社区可用的邮件列表，并根据您的兴趣加入某个邮件列表。\n\n希望以上信息对您有帮助，祝您在openEuler社区的参与之旅愉快！'}

*Optional 1:*   
同样可以使用我们部署的 `chatGLM` 来实现答案生成。

In [7]:
from langchain.llms import ChatGLM

# default endpoint_url for a local deployed ChatGLM api server
endpoint_url = "http://127.0.0.1:8000"

# direct access endpoint in a proxied environment
# os.environ['NO_PROXY'] = '127.0.0.1'

llm_chatglm = ChatGLM(
    endpoint_url=endpoint_url,
    max_token=80000,
    top_p=0.9,
    model_kwargs={"sample_model_args": False},
)

qa_chain = RetrievalQA.from_chain_type(llm_chatglm,retriever=vectorstore.as_retriever())
qa_chain({"query": question})

# turn on with_history only when you want the LLM object to keep track of the conversation history
# and send the accumulated context to the backend model api, which make it stateful. By default it is stateless.
# llm.with_history = True

{'query': '如何参与社区活动?',
 'result': 'To participate in community activities, you can follow these steps:\n\n1. Submit requirements or bugs: You can submit requirements or bugs to the openEuler community through the openEuler website, code repository, or email list.\n2. 修正 bugs: As a developer, you can contribute to the openEuler community by fixing bugs. You can also participate in the openEuler code repository to contribute new code or fix existing code.\n3. 贡献 software包： To help the openEuler community, you can contribute software包 by finding missing software包 or fixing bugs in the openEuler code repository.\n\nOverall, there are various ways to participate in the openEuler community, and the best way to do so will depend on your interests and experience.'}

*Optional 2:*   
在部分场景中我们可能需要对`chain`中涉及的`prompt`进行调整，新的`prompt`可以通过参数传递给对应的`chain`

In [8]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)
result = qa_chain({"query": question})
result["result"]

'要参与openEuler社区活动，您可以通过以下三种方式之一：1. 提交需求或bug，将在使用openEuler过程中发现的问题提出来。2. 修复bug，以开发者的身份参与社区，解决问题。3. 贡献软件包，帮助openEuler提供更丰富的功能。感谢您的提问！'

*Optional 3:*   
在基于文档生成答案时受限于模型能力，准确率差异化较大，且文档切片可能导致关键信息不完整，因此在问答中提供文档源和链接可有效提升问答效果，保障信息准确性。`RetrievalQAWithSourcesChain` 提供了这部分能力。

In [13]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_chain = RetrievalQAWithSourcesChain.from_chain_type(llm,retriever=vectorstore.as_retriever())

result = qa_chain({"question": question})
result

{'question': '如何参与社区活动?',
 'answer': '参与社区活动有三种方式：\n1. 提交需求或bug：在使用openEuler的过程中，发现问题或bug后，可以在社区中提出并讨论。\n2. 修正bug：以开发者的身份参与社区，解决问题和修复bug。\n3. 贡献软件包：发现openEuler缺失某个软件包时，可以帮助补充该软件包，为openEuler提供更丰富的功能。\n\n参与社区活动的具体方法可以参考以下链接：\n- openEuler官网：https://openeuler.org/openEuler\n- openEuler代码仓：https://gitee.com/openeuler/openEuler\n- 软件包仓：https://gitee.com/src-op\n\n在openEuler社区中，可以通过邮件列表和视频会议等方式进行交流和沟通。具体步骤可以参考以下链接：\n- 社区可用的邮件列表：https://www.openeuler.org/zh/community/mailing-list/\n\n参与社区活动前，需要首先签署CLA协议。根据参与身份的不同，可以选择签署个人CLA、员工CLA或企业CLA。协议签署地址为：https://clasign.osinfra.cn/sign/gitee_openeuler-1611298811283968340\n\n安装openEuler并开始贡献的详细步骤可以参考链接：[下载安装openEuler]\n\n',
 'sources': '- https://www.openeuler.org/zh/blog/20230630-newcomer/0630-newcomer.html'}

*Optional 4:*   
检索到的文档可以通过几种不同的方式输入到LLM中进行答案提炼: `stuff`, `refine`, `map-reduce`, and `map-rerank`.
stuff: 将列表中文档的内容均作为上下文传入。
Refine: 通过循环输入列表中文档并迭代更新其答案。
map-reduce: 首先对每个文档分别应用LLM链（Map步骤），将链的输出作为一个新文档。然后，它将所有新文档传递给一个单独的合并文档链以获得单一输出（Reduce步骤）。
map-rerank: 对每个文档运行prompt，不仅尝试完成任务，还会给出答案的置信度评分。返回最高得分的回答。

可以通过 `chain_type` 参数为`RetrievalQA` 设置文档输入模式.

In [10]:
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever(),
                                       chain_type="stuff")
result = qa_chain({"query": question})
print(result)

{'query': '如何参与社区活动?', 'result': '参与openEuler社区活动有以下几个步骤：\n\n1. 签署CLA协议：首先，您需要签署CLA协议，才能参与社区贡献。根据您的参与身份，选择签署个人CLA、员工CLA或企业CLA。协议的签署地址是：https://clasign.osinfra.cn/sign/gitee_openeuler-1611298811283968340\n\n2. 下载安装openEuler：请参考[下载安装openEuler]开始您的贡献。\n\n3. 提交需求或bug：在使用openEuler的过程中，如果发现了问题或有需求，您可以在社区中提交这些问题。您可以访问openEuler官网（https://openeuler.org/openEuler）并在其中找到相应的邮件列表，根据您的兴趣加入某个邮件列表，然后在邮件列表中提出您的问题或需求。\n\n4. 修正bug：如果您是开发者，您可以通过修正bug来参与社区。在openEuler的代码仓（https://gitee.com/openeuler/openEuler）中，您可以找到需要修复的bug，并提交您的修复代码。\n\n5. 贡献软件包：如果您发现openEuler缺失了某个软件包，您可以帮助openEuler补充这个软件包。您可以在openEuler的软件包仓（https://gitee.com/src-op）中查找相关信息，并提交您的贡献。\n\n希望以上步骤能帮助您参与openEuler社区活动。'}


**Step 6. Conversation**  
为了进行对话，`chain`需要能够引用过去的交流记录。`Memory` 使我们能够实现这一点。为了保留历史聊天记录，我们可以指定一个内存缓冲区来跟踪对话的输入和输出。在` ConversationalRetrievalChain`中使用 `memory`中的内容。

In [11]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

retriever = vectorstore.as_retriever()
chat = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

result = chat({"question": "openeuler是什么？"})
result['answer']

# The Memory buffer has context to resolve "openeuler" in the below question.
result = chat({"question": "如何参与它的活动？"})
result['answer']

'要参与openEuler的活动，您可以按照以下步骤进行：\n\n1. 首先，您需要签署openEuler的CLA协议。根据您的参与身份，选择签署个人CLA、员工CLA或企业CLA。您可以访问https://clasign.osinfra.cn/sign/gitee_openeuler-1611298811283968340来签署协议。\n\n2. 安装openEuler。您可以参考[下载安装openEuler]的指南来开始您的贡献。\n\n3. 参与社区的讨论和交流。openEuler社区有邮件列表和视频会议等交流方式。您可以访问https://www.openeuler.org/zh/community/mailing-list/来找到可用的邮件列表，并根据您的兴趣加入某个邮件列表。\n\n4. 根据您的兴趣和能力，选择参与的方式。您可以提交需求或bug，为社区修正bug，或贡献软件包。具体的参与方法可以在openEuler官网（https://openeuler.org/openEuler）、openEuler代码仓（https://gitee.com/openeuler/openEuler）和软件包仓（https://gitee.com/src-op）上找到。\n\n希望以上信息对您有帮助，祝您在openEuler社区的参与之旅愉快！'